In [2]:
import sys
sys.path.append('C:\\Users\\Besh Vignesh\\anaconda3\\envs\\dl\\Lib\\site-packages')

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from wordcloud import WordCloud


import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from multiprocessing import Pool
%matplotlib inline


In [5]:
with open('picklefile/prepare_data_set.pkl', 'rb') as data:
    df = pickle.load(data)

In [6]:
df['sentiment']=df['sentiment'].astype('int')

In [7]:
df.head()

,content,rating,sentiment,text length
0,This kindle is light and easy to use especiall...,5.0,1,63
1,Didnt know how much i'd use a kindle so went f...,4.0,1,107
2,I am 100 happy with my purchase. I caught it o...,5.0,1,757
3,Solid entry level Kindle. Great for kids. Gift...,5.0,1,176
4,This make an excellent ebook reader. Don't exp...,5.0,1,158


In [8]:
df.shape

(9596, 4)

In [551]:
df.iloc[4]['content']

"This make an excellent ebook reader. Don't expect much from this device except to read basic ebooks. The good thing is it's cheap and good to read in the sun."

In [10]:
df['sentiment'].value_counts()

0    4910
1    4686
Name: sentiment, dtype: int64

In [11]:
X=df.drop(['sentiment','text length'],axis=1)
y=df['sentiment']

In [12]:
X.shape

(9596, 2)

In [13]:
y.shape

(9596,)

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.4.1'

## RNN Model 

In [552]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [553]:
messages=X.copy()

In [554]:
messages['content'][1]

"Didnt know how much i'd use a kindle so went for the lower end. im happy with it, even if its a little dark"

In [555]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Besh
[nltk_data]     Vignesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [664]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['content'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069


3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728


5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368


6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021


8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631
8632
8633
8634
8635
8636
8637
8638
8639
8640
8641
8642
8643
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8657
8658
8659
8660
8661
8662
8663
8664
8665
8666
8667
8668
8669
8670
8671
8672
8673
8674
8675
8676


In [665]:
corpus

['kindl light easi use especi beach',
 'didnt know much use kindl went lower end im happi even littl dark',
 'happi purchas caught sale realli good price normal real book person year old love rip page kindl prevent extrem portabl fit better purs giant book load lot book finish one start anoth without go store serv need pick one paperwhit price unbeat differ could see one backlit simpl book light dollar tree solv issu second kindl first old keyboard model put fell love keyboard lol like last',
 'solid entri level kindl great kid gift kid friend love use read ipad batteri good higher model bit better',
 'make excel ebook reader expect much devic except read basic ebook good thing cheap good read sun',
 'order daughter black paperwhit love read quit bit larger book drive crazi hold lay want take book vacat lug around thick paperback throw bag read anywher light weight easi use batteri life last week make around awesom devic daughter public school offer onlin librari e reader download book

In [666]:
voc_size=1000
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[737, 460, 124, 603, 417, 598],
 [593, 898, 266, 603, 737, 62, 701, 859, 695, 130, 446, 456, 609],
 [130,
  507,
  889,
  426,
  181,
  402,
  244,
  705,
  545,
  537,
  904,
  801,
  685,
  548,
  861,
  958,
  737,
  980,
  450,
  974,
  651,
  1,
  758,
  71,
  537,
  590,
  176,
  537,
  26,
  738,
  26,
  566,
  570,
  280,
  4,
  170,
  199,
  629,
  738,
  949,
  244,
  257,
  46,
  818,
  41,
  738,
  68,
  563,
  537,
  460,
  337,
  407,
  53,
  998,
  40,
  737,
  919,
  685,
  640,
  368,
  481,
  228,
  548,
  640,
  754,
  130,
  459],
 [316,
  922,
  802,
  737,
  84,
  229,
  167,
  229,
  551,
  548,
  603,
  852,
  633,
  896,
  402,
  986,
  368,
  721,
  1],
 [311,
  646,
  198,
  274,
  100,
  266,
  753,
  60,
  852,
  662,
  198,
  402,
  682,
  669,
  402,
  852,
  900],
 [940,
  960,
  249,
  949,
  548,
  852,
  479,
  721,
  604,
  537,
  703,
  205,
  317,
  983,
  79,
  908,
  537,
  860,
  528,
  93,
  704,
  389,
  20,
  615,
  852,
  928,
  460,
  126,

In [667]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[  0   0   0 ... 603 417 598]
 [  0   0   0 ... 446 456 609]
 [563 537 460 ... 754 130 459]
 ...
 [  0   0   0 ... 957 748 661]
 [  0   0   0 ... 147 499 918]
 [  0   0   0 ... 408 430 100]]


In [668]:
embedded_docs[140]


array([  0,   0,   0,   0,   0,   0,   0, 548, 301, 952,  98, 561, 849,
       186,  29, 393, 434,   6, 178, 556])

In [669]:
## Creating model
embedding_vector_features=60
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add((LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (None, 20, 60)            60000     
_________________________________________________________________
dropout_60 (Dropout)         (None, 20, 60)            0         
_________________________________________________________________
lstm_31 (LSTM)               (None, 100)               64400     
_________________________________________________________________
dropout_61 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 1)                 101       
Total params: 124,501
Trainable params: 124,501
Non-trainable params: 0
_________________________________________________________________
None


In [670]:
len(embedded_docs),X.shape

(9596, (9596, 2))

In [671]:
len(embedded_docs),y.shape

(9596, (9596,))

In [672]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [673]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=42)

In [674]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30,batch_size=64)

Epoch 1/30
113/113 [==============================] - 4s 25ms/step - loss: 0.5557 - accuracy: 0.6973 - val_loss: 0.2511 - val_accuracy: 0.9033
Epoch 2/30
113/113 [==============================] - 2s 20ms/step - loss: 0.2243 - accuracy: 0.9159 - val_loss: 0.2375 - val_accuracy: 0.9112
Epoch 3/30
113/113 [==============================] - 2s 20ms/step - loss: 0.2010 - accuracy: 0.9259 - val_loss: 0.2243 - val_accuracy: 0.9158
Epoch 4/30
113/113 [==============================] - 3s 27ms/step - loss: 0.1617 - accuracy: 0.9400 - val_loss: 0.2130 - val_accuracy: 0.9187
Epoch 5/30
113/113 [==============================] - 3s 27ms/step - loss: 0.1500 - accuracy: 0.9438 - val_loss: 0.2250 - val_accuracy: 0.9204
Epoch 6/30
113/113 [==============================] - 3s 30ms/step - loss: 0.1396 - accuracy: 0.9520 - val_loss: 0.2251 - val_accuracy: 0.9179
Epoch 7/30
113/113 [==============================] - 3s 23ms/step - loss: 0.1279 - accuracy: 0.9541 - val_loss: 0.2348 - val_accuracy: 0.9191

In [675]:
model.evaluate(X_test,y_test)

75/75 [==============================] - 0s 3ms/step - loss: 0.4754 - accuracy: 0.9158


[0.4753859043121338, 0.9157982468605042]

In [676]:
y_pred=model.predict_classes(X_test)

In [677]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [678]:
confusion_matrix(y_test,y_pred)

array([[1114,   95],
       [ 107, 1083]], dtype=int64)

In [679]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      1209
           1       0.92      0.91      0.91      1190

    accuracy                           0.92      2399
   macro avg       0.92      0.92      0.92      2399
weighted avg       0.92      0.92      0.92      2399



In [680]:
print('Accuracy:{} %'.format(accuracy_score(y_test,y_pred)*100))

Accuracy:91.57982492705294 %


## Sample Prediction

In [902]:
text_to_predict ="""Works great with my Nikon cameras. I do shoot a lot of wildlife so the storage capacity really helps & the 150MB/sec speed is fantastic to shoot action."""

In [903]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus1 = []
review_to_predict = re.sub('[^a-zA-Z]', ' ', text_to_predict)
review_to_predict = review_to_predict.lower()
review_to_predict = review_to_predict.split() 
Lemmatizer = WordNetLemmatizer() 
review_to_predict = [Lemmatizer.lemmatize(w) for w in review_to_predict]
review_to_predict = [ps.stem(word) for word in review_to_predict if not word in stopwords.words('english')]
review_to_predict = ' '.join(review_to_predict)
corpus1.append(review_to_predict)

In [904]:
print(len(text_to_predict))

152


In [905]:
if len(text_to_predict)<500 and len(text_to_predict)>200:
    voc_size1=500
elif len(text_to_predict)<200:
    voc_size1=200
elif len(text_to_predict)>500:
    voc_size1=1000
onehot_repr1=[one_hot(words,voc_size1)for words in corpus1] 


In [906]:
sent_length1=20
embedded_docs1=pad_sequences(onehot_repr1,padding='pre',maxlen=sent_length1)

In [907]:
prediction = model.predict_classes(embedded_docs1)
prediction

array([[1]])

In [908]:
if prediction ==1:
    print("Thank You for your feedback!😊 ")
else:
    print("Sorry for the inconvenience. Our support staff will contact you and sort out the problem.")

Thank You for your feedback!😊 
